# Demonstrate the privacy models

In [1]:
import pickle
from tqdm import tqdm
import torch
import numpy as np
from main import init_seed # used by dataloaders
from time import sleep

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

In [2]:
with open('resnet.pkl', 'rb') as f:
    resnet = pickle.load(f).cuda()
with open('unet.pkl', 'rb') as f:
    unet = pickle.load(f).cuda()
    
with open('test_action.pkl', 'rb') as f:
    test_action = pickle.load(f)
# with open('test_privacy.pkl', 'rb') as f:
#     test_privacy = pickle.load(f)

In [3]:
d = next(iter(test_action))
d[0] = d[0].to('cuda')
d[0].shape

/home/thomas/.conda/envs/skele/lib/python3.9/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


torch.Size([64, 3, 300, 25, 2])

In [4]:
res_ = resnet.forward(d[0])
unet_ = resnet.forward(d[0])
# res_,unet_

In [5]:
res_[1].shape

torch.Size([3, 300, 25, 2])

In [3]:
del test_action#, res_, unet_, d,

# Function to convert their data to the way used by RF and SGD

In [7]:
from data_gen.rotation import *
import data_gen.rotation
from data_gen.preprocess import pre_normalization
from data_gen.ntu_gendata import read_xyz

In [8]:
r = read_xyz('/mnt/c/Users/Carrt/LocalCode/Skeleton-anonymization/data/raw_data/ntu60/S001C001P001R001A001.skeleton')

In [9]:
r[0][0][0][0], r[1][0][0][0], r[2][0][0][0]

(0.2181153, 0.1725972, 3.785547)

In [10]:
def anonymizer_to_sgd(t, max_frames=300):
    # (300, 150)
    # [x:0,y:1,z:2][300][joints][actors]
    xyz,frames,joints,actors = t.shape
    # transpose to make loop simple
    # [frame][actors][joints][xyz]
    t = t.transpose(1,3,2,0)
    # make empty array
    frames = []
    
    joints_per_frame = xyz*joints*actors
    
    # crazy loop
    for frame in t:
        f = []
        for actor in frame:
            for joint in actor:
                for xyz in joint:
                    f.append(xyz)
        
        # Pad 0's to 150 joints (2 actors)
        if len(f) < joints_per_frame:
            f = np.pad(f, (0, joints_per_frame-len(f)), 'constant')
            
        frames.append(f)
        
    # to numpy array
    X = np.array(frames, dtype=np.float32)
    
    if X.shape[0] < max_frames:
        X = np.pad(X, ((0, max_frames-X.shape[0]), (0, 0)), 'constant')
        
    return X

X = anonymizer_to_sgd(r)
X.shape

(300, 150)

In [11]:
del X, r

# Load the validation data and anonymize it

In [4]:
with open('data/ntu120/xview/val_data_joint.npy', 'rb') as f:
    X = np.load(f)
with open('data/ntu120/xview/val_label.pkl', 'rb') as f:
    y = pickle.load(f)
# with open('data/ntu120/xview/train_data_joint.npy', 'rb') as f:
#     X2 = np.load(f)
# with open('data/ntu120/xview/train_label.pkl', 'rb') as f:
#     y2 = pickle.load(f)

In [5]:
# Dict to hold all the data
X_unet = {}
X_resnet = {}

def run_split(X, y):
    # Convert X to torch tensor
    X = torch.tensor(X)
    
    # Split data into batches
    batch_size = 512
    X = torch.split(X, batch_size, dim=0)
 
    i = 0
    with torch.no_grad():  # Use torch.no_grad() to reduce memory usage
        for batch in tqdm(X):
            # Move batch to cuda
            batch_ = batch.cuda()
    
            # Get the anonymized data
            unet_res = unet.forward(batch_)#.cpu().numpy()
            resnet_res = resnet.forward(batch_)#.cpu().numpy()
    
            # Loop through actors in batch
            for x in range(len(batch_)):
                # Get the actor
                actor = y[0][i][8:12]
                i += 1

                # If actor not in dict, add it
                if actor not in X_unet:
                    X_unet[actor] = []
                    X_resnet[actor] = []
                    
                # Attempted fix to running out of CUDA memory
           
                # Apply unet and resnet privacy models
                X_unet[actor].append(unet_res[x].cpu().numpy())
                X_resnet[actor].append(resnet_res[x].cpu().numpy())
      
                # If actor not in dict, add it
                # if actor not in X_unet:
                #     X_unet[actor] = np.zeros((10000,3,300,25,2))
                #     X_resnet[actor] = np.zeros((10000,3,300,25,2))
           
                # # Apply unet and resnet privacy models
                # ind = np.where(X_unet[actor][:,0,0,0,0] == 0)[0][0]
                # X_unet[actor][ind] = unet_res[x]
                # X_resnet[actor][ind] = resnet_res[x]
              
            
            # Clear GPU memory and CPU memory after each batch
            del batch, batch_
            del unet_res, resnet_res
            torch.cuda.empty_cache()
        del X
    
    # Trim zeros from each actor
#     for actor in X_unet:
#         ind = np.where(X_unet[actor][:,0,0,0,0] == 0)[0][0]
#         X_unet[actor] = X_unet[actor][:ind]
#         X_resnet[actor] = X_resnet[actor][:ind]

In [6]:
run_split(X, y)
del X, y
# run_split(X2, y2)
# del X2, y2

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [17:47<00:00, 14.24s/it]


In [7]:
# Save results to pickle
with open('X_unet.pkl', 'wb') as f:
    pickle.dump(X_unet, f)
with open('X_resnet.pkl', 'wb') as f:
    pickle.dump(X_resnet, f)
# with open('X2_unet.pkl', 'wb') as f:
#     pickle.dump(X_unet, f)
# with open('X2_resnet.pkl', 'wb') as f:
#     pickle.dump(X_resnet, f)